In [2]:
import requests
import pandas as pd
import math
from ratelimit import limits, sleep_and_retry
import pyodbc

In [3]:
# 5 calls per minute
CALLS = 5
RATE_LIMIT = 60

@sleep_and_retry
@limits(calls=CALLS, period=RATE_LIMIT)
def check_limit():
#Empty function just to check for calls to API
    return

In [4]:
def getToken():
    check_limit()
    r = requests.get('https://public-apis-api.herokuapp.com/api/v1/auth/token')
    response = r.json()
    myToken = response['token']
    return myToken

In [5]:
def getCategories(myToken):
    check_limit()
    URL = 'https://public-apis-api.herokuapp.com/api/v1/apis/categories?page={}'.format(1)
    head = {'Authorization': 'token {}'.format(myToken)}
    response = (requests.get(URL, headers=head)).json()
    if('error' in response.keys()):
        head = {'Authorization': 'token {}'.format(getToken())}
        response = (requests.get(URL, headers=head)).json()    
    count = response['count']
    pages = math.ceil(count/10)
    df = pd.DataFrame(columns=['categories'])
    for i in response['categories']:
        df = df.append({'categories': i}, ignore_index=True)    
    if(pages>1):
        j = 2
        while(j <= pages):
            URL = 'https://public-apis-api.herokuapp.com/api/v1/apis/categories?page={}'.format(j)
            response = (requests.get(URL, headers=head)).json()
            if('error' in response.keys()):
                head = {'Authorization': 'token {}'.format(getToken())}
                response = (requests.get(URL, headers=head)).json()
            
            for i in response['categories']:
                df = df.append({'categories': i}, ignore_index=True)
            j = j+1        
    return df                    

In [6]:
categories_df = getCategories(getToken())
categories_df.head(50)

,categories
0,Animals
1,Anime
2,Anti-Malware
3,Art & Design
4,Books
5,Business
6,Calendar
7,Cloud Storage & File Sharing
8,Continuous Integration
9,Cryptocurrency


In [7]:
URL = 'https://public-apis-api.herokuapp.com/api/v1/apis/entry?page={pageN}&category={categoryN}'.format(pageN=1, categoryN=(categories_df['categories'][0]))
head = {'Authorization': 'token {}'.format(getToken())}
response = (requests.get(URL, headers=head)).json()
data = pd.DataFrame(columns=['API', 'Description', 'Auth', 'HTTPS', 'Cors', 'Link', 'Category'])
for counter in range(len(response['categories'])):
    temp = pd.Series(response['categories'][counter], index = data.columns)
    data = data.append(temp, ignore_index=True)
    
for i in range(len(categories_df['categories'])):
    check_limit()
    print('Extracting data from category - {}'.format(categories_df['categories'][i]))
    pageNumber = 1
    if(i==0):
        pageNumber = pageNumber + 1
        URL = 'https://public-apis-api.herokuapp.com/api/v1/apis/entry?page={pageN}&category={categoryN}'.format(pageN=pageNumber, categoryN=(categories_df['categories'][i]))
        response = (requests.get(URL, headers=head)).json()
        for counter in range(len(response['categories'])):
            temp = pd.Series(response['categories'][counter], index = data.columns)
            data = data.append(temp, ignore_index=True)
    else:    
        URL = 'https://public-apis-api.herokuapp.com/api/v1/apis/entry?page={pageN}&category={categoryN}'.format(pageN=pageNumber, categoryN=(categories_df['categories'][i]))
        response = (requests.get(URL, headers=head)).json()
        if('error' in response.keys()):
            head = {'Authorization': 'token {}'.format(getToken())}
            response = (requests.get(URL, headers=head)).json()
        for counter in range(len(response['categories'])):
            temp = pd.Series(response['categories'][counter], index = data.columns)
            data = data.append(temp, ignore_index=True)
        pageNumber = pageNumber + 1
        count = response['count']
        pages = math.ceil(count/10)
        while(pageNumber <= pages):
            check_limit()
            URL = 'https://public-apis-api.herokuapp.com/api/v1/apis/entry?page={pageN}&category={categoryN}'.format(pageN=pageNumber, categoryN=(categories_df['categories'][i]))
            response = (requests.get(URL, headers=head)).json()
            if('error' in response.keys()):
                head = {'Authorization': 'token {}'.format(getToken())}
                response = (requests.get(URL, headers=head)).json()    
            for counter in range(len(response['categories'])):
                temp = pd.Series(response['categories'][counter], index = data.columns)
                data = data.append(temp, ignore_index=True)
            pageNumber = pageNumber + 1        

Extracting data from category - Animals
Extracting data from category - Anime
Extracting data from category - Anti-Malware
Extracting data from category - Art & Design
Extracting data from category - Books
Extracting data from category - Business
Extracting data from category - Calendar
Extracting data from category - Cloud Storage & File Sharing
Extracting data from category - Continuous Integration
Extracting data from category - Cryptocurrency
Extracting data from category - Currency Exchange
Extracting data from category - Data Validation
Extracting data from category - Development
Extracting data from category - Dictionaries
Extracting data from category - Documents & Productivity
Extracting data from category - Environment
Extracting data from category - Events
Extracting data from category - Finance
Extracting data from category - Food & Drink
Extracting data from category - Games & Comics
Extracting data from category - Geocoding
Extracting data from category - Government
Extra

In [8]:
data.shape

(525, 7)

In [9]:
data.head(20)

,API,Description,Auth,HTTPS,Cors,Link,Category
0,Cat Facts,Daily cat facts,,True,no,https://alexwohlbruck.github.io/cat-facts/,Animals
1,Cats,Pictures of cats from Tumblr,apiKey,True,unknown,https://docs.thecatapi.com/,Animals
2,Dogs,Based on the Stanford Dogs Dataset,,True,yes,https://dog.ceo/dog-api/,Animals
3,HTTPCat,Cat for every HTTP Status,,True,unknown,https://http.cat/,Animals
4,IUCN,IUCN Red List of Threatened Species,apiKey,False,unknown,http://apiv3.iucnredlist.org/api/v3/docs,Animals
5,Movebank,Movement and Migration data of animals,,True,unknown,https://github.com/movebank/movebank-api-doc,Animals
6,Petfinder,Adoption,OAuth,True,yes,https://www.petfinder.com/developers/v2/docs/,Animals
7,PlaceGOAT,Placeholder goat images,,True,unknown,https://placegoat.com/,Animals
8,RandomCat,Random pictures of cats,,True,yes,https://aws.random.cat/meow,Animals
9,RandomDog,Random pictures of dogs,,True,yes,https://random.dog/woof.json,Animals


In [10]:
server = '192.168.0.105' 
database = 'TestDB' 
username = 'Sa' 
password = 'Mssql@123' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in data.iterrows():
    cursor.execute("INSERT INTO dbo.Assignment (API ,Description ,Auth ,HTTPS ,Cors ,Link, Category) values(?,?,?,?,?,?,?)", row.API, row.Description, row.Auth, row.HTTPS, row.Cors, row.Link, row.Category)
cnxn.commit()
cursor.close()